In [7]:
import numpy as np
import pandas as pd

train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
submission=pd.read_csv('sample_submission.csv')



def year(date_time):
    return int(date_time[:4])

def month(date_time):
    return int(date_time[5:7])

#적용
train['year'] = train['date_time'].apply(year)
test['year'] = test['date_time'].apply(year)

train['month'] = train['date_time'].apply(month)
test['month'] = test['date_time'].apply(month)

#date_time 제거
train=train.drop(['date_time'], axis=1)
test=test.drop(['date_time'], axis=1)

X=train.drop(columns=['number_of_rentals', 'wind_direction'])
train['number_of_rentals'] = np.log(train['number_of_rentals'])  # 자연 로그
y=train['number_of_rentals']

from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, valid_idx in kf.split(X):
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]


from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_valid_scaled = sc.transform(X_valid)
X_test_scaled = sc.transform(test.drop(columns=['wind_direction']))


from sklearn.svm import SVR
model=SVR(kernel='rbf', C=5.0, epsilon=0.1, gamma=0.01
          )
model.fit(X_train_scaled, y_train)

y_train_pred = model.predict(X_train_scaled)
y_valid_pred = model.predict(X_valid_scaled)
y_test_pred = model.predict(X_test_scaled)

y_train_pred_original=np.exp(y_train_pred)
y_valid_pred_original=np.exp(y_valid_pred)
y_test_pred_original=np.exp(y_test_pred)

def nmae(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true))

train_nmae = nmae(np.exp(y_train), y_train_pred_original)
valid_nmae = nmae(np.exp(y_valid), y_valid_pred_original)

print(f"Train NMAE: {train_nmae:}")
print(f"Validation NMAE: {valid_nmae:}")

submission['number_of_rentals']=y_test_pred_original
submission.to_csv('SVR_ver4_submission.csv', index=False)


Train NMAE: 0.1609098224974924
Validation NMAE: 0.1967302039950036


In [ ]:
import os
import pandas as pd

folder_path = r"c:\Users\김은교\OneDrive\바탕 화면\BRAIN\Dacon-project\bicycle_rentals_prediction"
file_list = os.listdir(folder_path)

for filename in file_list:
    file_path = os.path.join(folder_path, filename)
    try:
        if filename.endswith(".txt"):
            with open(file_path, "r", encoding="utf-8") as file:
                content = file.read()
                print(content[:200])  # 앞부분만 출력

        elif filename.endswith(".csv"):
            df = pd.read_csv(file_path, encoding="utf-8")
            print(df.head())

        else:
            print(f"Skipping unsupported file type: {filename}")

    except PermissionError:
        print(f"Permission denied: {file_path}")
    except IsADirectoryError:
        print(f"Skipping directory: {file_path}")
    except UnicodeDecodeError:
        print(f"Encoding error: {file_path}")


In [3]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.5, 0.1, 1, 5, 10, 50],
    'epsilon': [0.1, 0.2, 0.3, 0.4, 0.5],
    'gamma': [0.01, 0.05, 0.1, 0.5, 'scale']
}

from sklearn.metrics import make_scorer

def NMAE(true, pred):
    return np.mean(np.abs(true - pred) / true)
NMAE_scorer = make_scorer(NMAE, greater_is_better=False) # NMAE가 작을수록 좋음

grid = GridSearchCV(SVR(kernel='rbf'), param_grid, scoring=NMAE_scorer, cv=5, n_jobs=1)
grid.fit(X_train_scaled, y_train)

print("Best params:", grid.best_params_)


Best params: {'C': 5, 'epsilon': 0.1, 'gamma': 0.05}
